In [ ]:
###################IGUAL AO DELIVERABLE####################

# !pip install pyshp
import pandas as pd
import pickle
from scipy.spatial import cKDTree
import numpy as np

def read_shapefile(shp_path):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' column holding
    the geometry information. This uses the pyshp package
    """
    import shapefile

    #read file, parse out the records and shapes
    sf = shapefile.Reader(shp_path)
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]

    #write into a dataframe
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)

    return df


from google.colab import drive
drive.mount('/content/drive')

df = read_shapefile('/content/drive/MyDrive/WDL/outdoor_inventory/Outdoor_Inventory_AV.shp')
reverse_coords_list = pd.read_pickle('/content/drive/MyDrive/WDL/reverse_loc.pickle')
df['address_road'] = [x.get('address').get('road') for x in reverse_coords_list]
df['concelho'] = [x.get('address').get('town') for x in reverse_coords_list]
df['freguesia'] = [x.get('address').get('village') if 'village' in x.get('address') else x.get('address').get('neighbourhood') for x in reverse_coords_list]
df['distrito'] = [x.get('address').get('county') for x in reverse_coords_list]
df['address_road'] = df['address_road'].fillna('')
df['is_national_road'] = df['address_road'].str.contains('EN')
df['is_highway'] = df['address_road'].str.contains('Auto')
df['is_city_center'] = (~df['is_national_road']) & (~df['is_highway']) & (df['address_road'] != '')


def find_neighbours_within_radius(xy, radius):
    tree = cKDTree(xy)
    within_radius = tree.query_ball_tree(tree, r=radius)
    return within_radius

def get_density_billboards(df):
    """
    Calculate number of billboards in a X km radius for each point
    """
    
    df_copy = df.copy()

    import numpy as np

    kms_per_radian = 6371.0088
    radius = 1.5
    neighbours_within_radius = find_neighbours_within_radius(
        np.radians(df_copy[['Y', 'X']].values), 
        radius/kms_per_radian
    )

    df_copy['nbr_points_around_billboard'] = [len(x) for x in neighbours_within_radius]

    return df_copy, neighbours_within_radius

df, neighbours_within_radius = get_density_billboards(df)
df['average_people_around_billboard'] = [np.mean(df.iloc[x].Average_Da) for x in neighbours_within_radius]
df['std_people_around_billboard'] = [np.std(df.iloc[x].Average_Da) for x in neighbours_within_radius]
df['diff_to_neighbourhood'] = df['Average_Da'] - df['average_people_around_billboard']

In [ ]:
df[["X", "Y"]].value_counts().to_frame('n_OutdoorAd').reset_index()

#Existe uma zona com 11 billboards e outra zona com 6. Depois existem várias zonas com 4, 3, 2 e 1 billboards. (Será que faz sentido por em forma de mapa ou assim? Para não se ver 1 a 1)

In [ ]:
#Zona com 11 billboards
df.loc[(df['X'] == -9.128028) & (df['Y'] == 38.620447)]

#As coordenadas de latitude e longitude com o maior número de billboards (11) correspondem à Autoestrada do Sul (também conhecida como A2) no concelho do Seixal,
#freguesia Cruz de Pau e distrito Setúbal. Apresenta outros 132 billboards "vizinhos".

In [ ]:
#Zona com 6 billboards
df.loc[(df['X'] == -9.167858) & (df['Y'] == 38.741668)]

#A praça Marechal Humberto Delgado no centro da cidade de Lisboa tem 6 billboards. Existe um total de 540 billboards num raio de 1.5km. 